In [ ]:
import torch
from dataloader import GraphDataset, GraphTextDataset, TextDataset, GraphTextInMDataset
import networkx as nx

from transformers import AutoTokenizer

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext')

In [ ]:
import os
import shutil
from tqdm import tqdm

source_directory = r"C:\Antoine\Study\Master 2 - MVA\ALTEGRAD\Challenge\Public\Public\data"
destination_directory = r"C:\Antoine\Study\Master 2 - MVA\ALTEGRAD\Altegrad-MVA-2023-2024\data"

def move_files(source_directory, destination_directory):
    # Make sure the destination directory exists, create it if not
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    # Get the total number of files to track progress
    total_files = sum([len(files) for _, _, files in os.walk(source_directory)])

    # Initialize the tqdm progress bar
    progress_bar = tqdm(total=total_files, desc="Moving files", unit="file")

    # Walk through the source directory and its subdirectories
    for root, _, files in os.walk(source_directory):
        for file_name in files:
            source_path = os.path.join(root, file_name)
            # Create the corresponding subdirectory structure in the destination
            relative_path = os.path.relpath(source_path, source_directory)
            destination_path = os.path.join(destination_directory, relative_path)

            destination_dir = os.path.dirname(destination_path)
            if not os.path.exists(destination_dir):
                os.makedirs(destination_dir)


            # Move the file
            shutil.move(source_path, destination_path)

            # Update the progress bar
            progress_bar.update(1)

    # Close the progress bar
    progress_bar.close()

In [ ]:
# move_files(source_directory, destination_directory)

In [ ]:
train_dir = r"data/processed/train/"
test_dir = r"data/processed/test/"

In [ ]:
sample = torch.load(train_dir + "data.pt")

In [ ]:
def load_datasets(tokenizer: AutoTokenizer):
    gt = np.load("./data/token_embedding_dict.npy", allow_pickle=True)[()]
    val_dataset = GraphTextInMDataset(
        root="./data/", gt=gt, split="val", tokenizer=tokenizer
    )
    train_dataset = GraphTextInMDataset(
        root="./data/", gt=gt, split="train", tokenizer=tokenizer
    )
    return val_dataset, train_dataset

val_dataset, train_dataset = load_datasets(tokenizer)

In [ ]:
sample = train_dataset[120]

In [ ]:
import textwrap

In [ ]:
def show_sample(sample):
    edges = sample.edge_index
    G = nx.DiGraph()

    # Add edges to the graph
    for i in range(len(edges[0])):
        G.add_edge(int(edges[0][i]), int(edges[1][i]))

    # Draw the graph
    pos = nx.kamada_kawai_layout(G)
    nx.draw(G, pos, with_labels=True, font_size=8, node_color='skyblue', node_size=200, edge_color='gray', linewidths=0.3, arrows=False)

    # Show the plot
    decoded_input = tokenizer.batch_decode(sample.input_ids, skip_special_tokens=True)[0]
    wrapped_text = textwrap.fill(decoded_input, width=70)

    plt.text(0, -1, wrapped_text, ha='center', va='center', fontsize=8, bbox=dict(facecolor='white', alpha=0.4))

In [ ]:
plt.figure(figsize=(15, 15))

plt.subplot(2, 2, 1)
show_sample(train_dataset[41])
plt.subplot(2, 2, 2)
show_sample(train_dataset[121])
plt.subplot(2, 2, 3)
show_sample(train_dataset[452])
plt.subplot(2, 2, 4)
show_sample(train_dataset[71])

In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext')

No sentence-transformers model found with name C:\Users\Antoine/.cache\torch\sentence_transformers\microsoft_BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext. Creating a new one with MEAN pooling.


In [ ]:
decoded_input = tokenizer.batch_decode(train_dataset[41].input_ids, skip_special_tokens=True)[0]
print(decoded_input)
embedding = model.encode(
    decoded_input
)
print(embedding.shape)